# 실전 문제 해결 (부족한 데이터셋)

In [1]:
import tensorflow as tf
import numpy as np
import random

from imblearn.over_sampling import BorderlineSMOTE

## 하이퍼 파라미터

In [2]:
EPOCHS = 50

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## 데이터셋 준비 (불균형한 데이터셋)

In [4]:
cifar10 = tf.keras.datasets.cifar10 # 32x32x3

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# TODO: 학습 데이터를 Imbalanced small dataset으로 변형하기
x_train_small = list()
y_train_small = list()
for x, y in zip(x_train, y_train):
    if (y == 0 and random.randint(0, 100) < 10) or y == 1:
        x_train_small.append(x[:])
        y_train_small.append(y)

x_test_small = list()
y_test_small = list()
for x, y in zip(x_test, y_test):
    if y == 0 or y == 1:
        x_test_small.append(x[:])
        y_test_small.append(y)
        
x_train = np.stack(x_train_small, axis=0)
y_train = np.stack(y_train_small, axis=0)

x_test = np.stack(x_test_small, axis=0)
y_test = np.stack(y_test_small, axis=0)
    
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습 (불균형한 데이터셋)

In [5]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                             tf.keras.metrics.Precision(name='precision'),
                             tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/50
171/171 [==============================] - 4s 17ms/step - loss: 0.4672 - accuracy: 0.9062 - precision: 0.9278 - recall: 0.9734 - val_loss: 0.6187 - val_accuracy: 0.6980 - val_precision: 0.6304 - val_recall: 0.9570
Epoch 2/50
171/171 [==============================] - 3s 15ms/step - loss: 0.2311 - accuracy: 0.9249 - precision: 0.9347 - recall: 0.9870 - val_loss: 1.1845 - val_accuracy: 0.5765 - val_precision: 0.5419 - val_recall: 0.9900
Epoch 3/50
171/171 [==============================] - 3s 16ms/step - loss: 0.2254 - accuracy: 0.9283 - precision: 0.9404 - recall: 0.9842 - val_loss: 0.8613 - val_accuracy: 0.6455 - val_precision: 0.5866 - val_recall: 0.9860
Epoch 4/50
171/171 [==============================] - 3s 15ms/step - loss: 0.2064 - accuracy: 0.9304 - precision: 0.9425 - recall: 0.9840 - val_loss: 0.7059 - val_accuracy: 0.6650 - val_precision: 0.6002 - val_recall: 0.9880
Epoch 5/50
171/171 [==============================] - 3s 15ms/step - loss: 0.2002 - accuracy: 0.9326

171/171 [==============================] - 3s 16ms/step - loss: 0.0923 - accuracy: 0.9637 - precision: 0.9728 - recall: 0.9880 - val_loss: 0.5915 - val_accuracy: 0.8060 - val_precision: 0.7287 - val_recall: 0.9750
Epoch 38/50
171/171 [==============================] - 3s 15ms/step - loss: 0.0762 - accuracy: 0.9710 - precision: 0.9777 - recall: 0.9910 - val_loss: 0.6919 - val_accuracy: 0.7900 - val_precision: 0.7101 - val_recall: 0.9800
Epoch 39/50
171/171 [==============================] - 3s 15ms/step - loss: 0.0853 - accuracy: 0.9650 - precision: 0.9751 - recall: 0.9870 - val_loss: 0.7745 - val_accuracy: 0.7720 - val_precision: 0.6913 - val_recall: 0.9830
Epoch 40/50
171/171 [==============================] - 3s 15ms/step - loss: 0.0787 - accuracy: 0.9705 - precision: 0.9775 - recall: 0.9906 - val_loss: 0.5443 - val_accuracy: 0.8295 - val_precision: 0.7572 - val_recall: 0.9700
Epoch 41/50
171/171 [==============================] - 3s 16ms/step - loss: 0.0770 - accuracy: 0.9718 - prec

## 데이터셋 준비 (BorderlineSMOTE)

In [6]:
# TODO: BorderlineSMOTE 적용하기
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3])).astype(np.float32)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2] * x_test.shape[3])).astype(np.float32)

smote = BorderlineSMOTE()
x_train, y_train = smote.fit_resample(x_train, y_train)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Keras API 모델 학습 (BorderlineSMOTE)

In [7]:
model = MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                             tf.keras.metrics.Precision(name='precision'),
                             tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/50
313/313 [==============================] - 6s 15ms/step - loss: 0.5722 - accuracy: 0.7618 - precision: 0.7789 - recall: 0.7312 - val_loss: 0.5040 - val_accuracy: 0.7585 - val_precision: 0.9033 - val_recall: 0.5790
Epoch 2/50
313/313 [==============================] - 4s 14ms/step - loss: 0.3879 - accuracy: 0.8365 - precision: 0.8625 - recall: 0.8006 - val_loss: 0.3834 - val_accuracy: 0.8395 - val_precision: 0.8511 - val_recall: 0.8230
Epoch 3/50
313/313 [==============================] - 4s 14ms/step - loss: 0.2955 - accuracy: 0.8848 - precision: 0.9123 - recall: 0.8514 - val_loss: 0.5104 - val_accuracy: 0.7965 - val_precision: 0.7276 - val_recall: 0.9480
Epoch 4/50
313/313 [==============================] - 5s 14ms/step - loss: 0.2742 - accuracy: 0.8934 - precision: 0.9169 - recall: 0.8652 - val_loss: 0.4212 - val_accuracy: 0.8295 - val_precision: 0.7960 - val_recall: 0.8860
Epoch 5/50
313/313 [==============================] - 5s 15ms/step - loss: 0.2336 - accuracy: 0.9102

313/313 [==============================] - 4s 14ms/step - loss: 0.0335 - accuracy: 0.9887 - precision: 0.9910 - recall: 0.9864 - val_loss: 0.7650 - val_accuracy: 0.8700 - val_precision: 0.8234 - val_recall: 0.9420
Epoch 38/50
313/313 [==============================] - 4s 14ms/step - loss: 0.0290 - accuracy: 0.9909 - precision: 0.9936 - recall: 0.9882 - val_loss: 0.8987 - val_accuracy: 0.8510 - val_precision: 0.7920 - val_recall: 0.9520
Epoch 39/50
313/313 [==============================] - 4s 14ms/step - loss: 0.0291 - accuracy: 0.9904 - precision: 0.9932 - recall: 0.9876 - val_loss: 1.3216 - val_accuracy: 0.8125 - val_precision: 0.7337 - val_recall: 0.9810
Epoch 40/50
313/313 [==============================] - 4s 14ms/step - loss: 0.0265 - accuracy: 0.9908 - precision: 0.9928 - recall: 0.9888 - val_loss: 0.9568 - val_accuracy: 0.8370 - val_precision: 0.7731 - val_recall: 0.9540
Epoch 41/50
313/313 [==============================] - 4s 14ms/step - loss: 0.0405 - accuracy: 0.9872 - prec